### Objective

Using spotipy API to retrieve songs from various playlists.  
**Third step is retrieving the song id's from the playlists.**

In order to not send too many API request of same nature, I am saving the intermediate result. 

*Output: 3_Song_Ids.csv*, columns 'category_id', 'category_name', 'playlist_id', 'playlist_name', 'song_id', 'song_artist', song_title

In [ ]:
import pandas as pd
from spotipy_util import retrieve_sp

> retrieve categories

In [ ]:
playlist_df = pd.read_csv('./cache/3_Playlist_Ids.csv')
display(playlist_df.head(1))
display(playlist_df.shape)

> connect to api

In [ ]:
sp = retrieve_sp()

> retrieve all song id's, artists, titles from each playlist

In [21]:
playlist_df = playlist_df.head(2)
playlist_df

,category_id,category_name,playlist_id,playlist_name
0,0JQ5DAqbMKFz6FAsUtgAab,New Releases,37i9dQZF1DWUW2bvSkjcJ6,New Music Friday Deutschland
1,0JQ5DAqbMKFQ00XGBls6ym,Hip-Hop,37i9dQZF1DX36edUJpD76c,Modus Mio


In [22]:
all_songs = []

# Iterate over each playlist
for idx, row in playlist_df.iterrows():
    playlist_id = row['playlist_id']
    category_id = row['category_id']
    category_name = row['category_name']
    
    # Get playlist details
    playlist = sp.playlist(playlist_id)
    playlist_name = playlist['name']
    
    # Get tracks for the current playlist
    tracks = sp.playlist_tracks(playlist_id, fields='items.track.id,items.track.artists,items.track.name,total')
    
    # Check if tracks is not None and 'items' in tracks
    if tracks is not None and 'items' in tracks:
        # Iterate over each track and append its data to the list
        for item in tracks['items']:
            # Check if 'track' key exists in the item and it's not None
            if 'track' in item and item['track'] is not None:
                # Check if 'id' key exists in the track
                if 'id' in item['track']:
                    song_id = item['track']['id']
                    song_title = item['track']['name']
                    song_artist = ', '.join([artist['name'] for artist in item['track']['artists']])
                    all_songs.append({'category_id': category_id, 'category_name': category_name, 'playlist_id': playlist_id, 'playlist_name': playlist_name, 'song_id': song_id, 'song_artist': song_artist, 'song_title': song_title})


In [24]:
songs_df = pd.DataFrame(all_songs)
songs_df

,category_name,playlist_id,track_id,track_name,track_artists
0,New Releases,37i9dQZF1DWUW2bvSkjcJ6,7221xIgOnuakPdLqT0F3nP,I Had Some Help (Feat. Morgan Wallen),"Post Malone, Morgan Wallen"
1,New Releases,37i9dQZF1DWUW2bvSkjcJ6,4uDyEmWYoyfi2azbGS4IVs,nimm mit,"Paula Hartmann, makko"
2,New Releases,37i9dQZF1DWUW2bvSkjcJ6,1TFK9oczePSWgNtWRWlUoI,Balla Balla,"Ski Aggu, Marlon Hoffstadt"
3,New Releases,37i9dQZF1DWUW2bvSkjcJ6,7MYLHLCf0YWR4TVs3owDny,Fire - Official UEFA EURO 2024 Song,"MEDUZA, OneRepublic, Leony"
4,New Releases,37i9dQZF1DWUW2bvSkjcJ6,6df3If2RzcZ0CP79sVEuSe,Cabrio,"Olexesh, Bonez MC"
...,...,...,...,...,...
128,Hip-Hop,37i9dQZF1DX36edUJpD76c,7AGtu4Lg1qMeky9XCkWu9h,Geständnis,Samra
129,Hip-Hop,37i9dQZF1DX36edUJpD76c,2IljGIvJWtgjvG4h8WUh8E,SMS,Teven
130,Hip-Hop,37i9dQZF1DX36edUJpD76c,1kQKmatH0qVvnT05kJ0o4d,Uludağ und Sorgen,"Paula Hartmann, Symba"
131,Hip-Hop,37i9dQZF1DX36edUJpD76c,7G6CXD57a8mIUFoOrmMhU7,Ella,Monet192


In [ ]:
playlist_id = playlist_df['playlist_id'][1]
playlist_id
    

> there is a total off 55 categories  
> retrieve up to 5 playlists per category